In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open('data/names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
len(words)

32033

In [7]:
# build the vocabulary of characters and map to integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [120]:
# build the dataset

block_size = 3 # context length, how many characters to predict the next one
X, Y = [], []
for w in words:
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        context = context[1:] + [ix]

X = torch.tensor(X) # input dataset, previous 3 characters of context, padded with '.'
Y = torch.tensor(Y) # labels, next character

In [121]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [20]:
C = torch.randn((27, 2))

In [28]:
# this method is more in line with considering the embedding table
# the "first layer" of the neural network, converting our inputs
# to one-hot and then indexing into the embedding that way.

# however, it's much faster to just directly index, so instead we will
# manually pre-index into the embedding table and then consider the
# embedded raw data as the input to our neural net.

F.one_hot(torch.tensor(5), num_classes=27).float() @ C

tensor([-0.2067, -0.9111])

In [29]:
C[5]

tensor([-0.2067, -0.9111])

In [34]:
# we can fetch embeddings for every character in X just by directly indexing
C[X].shape

torch.Size([32, 3, 2])

In [35]:
emb = C[X]

In [36]:
W1 = torch.randn((2 * block_size, 100)) # 2 inputs per character of context
b1 = torch.randn(100)

In [38]:
# incompatible sizes, we need to concatenate the 3,2 portions of our embeddings
emb.shape, W1.shape

(torch.Size([32, 3, 2]), torch.Size([6, 100]))

In [45]:
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], 1).shape


torch.Size([32, 6])

In [48]:
# or, to do WLOG w/r/t block_size
torch.cat(torch.unbind(emb, 1), 1).shape

torch.Size([32, 6])

In [51]:
a = torch.arange(18)
a

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

In [52]:
a.shape

torch.Size([18])

In [54]:
# view is extremely efficient in pytorch
# the underlying storage is not affected at all, only organizational info about
# how the storage is represented to python.
# storage offsets, strides, and shapes are changed so that identical 
# sequences of bytes can represent different shapes of data.
# http://blog.ezyang.com/2019/05/pytorch-internals/
a.view(2, 9)

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8],
        [ 9, 10, 11, 12, 13, 14, 15, 16, 17]])

In [65]:
emb.shape

torch.Size([32, 3, 2])

In [70]:
emb.view(32, 6), emb[3, 0], emb[3, 1], emb[3, 2]

(tensor([[ 1.1152, -0.3656,  1.1152, -0.3656,  1.1152, -0.3656],
         [ 1.1152, -0.3656,  1.1152, -0.3656, -0.2067, -0.9111],
         [ 1.1152, -0.3656, -0.2067, -0.9111,  0.1083,  1.6349],
         [-0.2067, -0.9111,  0.1083,  1.6349,  0.1083,  1.6349],
         [ 0.1083,  1.6349,  0.1083,  1.6349,  0.8300, -0.3442],
         [ 1.1152, -0.3656,  1.1152, -0.3656,  1.1152, -0.3656],
         [ 1.1152, -0.3656,  1.1152, -0.3656, -0.5170,  0.0476],
         [ 1.1152, -0.3656, -0.5170,  0.0476,  1.1058, -0.1189],
         [-0.5170,  0.0476,  1.1058, -0.1189,  0.8579,  1.3922],
         [ 1.1058, -0.1189,  0.8579,  1.3922, -0.5378, -0.6277],
         [ 0.8579,  1.3922, -0.5378, -0.6277,  0.8579,  1.3922],
         [-0.5378, -0.6277,  0.8579,  1.3922,  0.8300, -0.3442],
         [ 1.1152, -0.3656,  1.1152, -0.3656,  1.1152, -0.3656],
         [ 1.1152, -0.3656,  1.1152, -0.3656,  0.8300, -0.3442],
         [ 1.1152, -0.3656,  0.8300, -0.3442, -0.5378, -0.6277],
         [ 0.8300, -0.344

In [76]:
h = torch.tanh(emb.view(-1, block_size * 2) @ W1 + b1) 
# b1 is broadcasted so the same bias is added to every row of output, which is what we want

In [77]:
h.shape

torch.Size([32, 100])

In [78]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [80]:
logits = h @ W2 + b2

In [82]:
logits.shape

torch.Size([32, 27])

In [83]:
counts = logits.exp()
prob = counts / counts.sum(1, keepdims=True)

In [86]:
prob.shape

torch.Size([32, 27])

In [93]:
loss = -prob[torch.arange(32), Y].log().mean()

In [163]:
# clean up
g = torch.Generator().manual_seed(2147483647)
emb_size = 2
hidden_size = 100
C = torch.randn((27, emb_size), generator=g)
W1 = torch.randn((emb_size * block_size, hidden_size), generator=g)
b1 = torch.randn(hidden_size, generator=g)
W2 = torch.randn((hidden_size, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [164]:
sum(p.nelement() for p in parameters)

3481

In [165]:
emb = C[X] # 32x2x2
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # -1 means torch will infer the size - output is 32x100
logits = h @ W2 + b2 # 32x27

In [166]:
# this is much more efficient - avoids all the intermediate allocation
# and backward pass is much more efficient without all the intermediate operations
loss = F.cross_entropy(logits, Y) # identical!
loss

tensor(19.5052)

In [108]:
logits = torch.tensor([-100, -3, 0, 100])
counts = logits.exp()
probs = counts / counts.sum()
probs 
# bad! our custom rolled cross_entropy results in a 
# nan because we run out of dynamic range on high inputs passed to exp()

tensor([0., 0., 0., nan])

In [112]:
# any fixed offset does not change the output, so pytorch internally does something like this,
# subtracting the highest input value from the entire vector to avoid any overflow.
logits = torch.tensor([-100, -3, 0, 100]) - 100
counts = logits.exp()
probs = counts / counts.sum()
probs

tensor([0.0000e+00, 1.4013e-45, 3.7835e-44, 1.0000e+00])

In [167]:
for p in parameters:
    p.requires_grad = True

In [130]:
for i in range(10):
    # forward pass
    emb = C[X] # 32x2x2
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # -1 means torch will infer the size - output is 32x100
    logits = h @ W2 + b2 # 32x27
    loss = F.cross_entropy(logits, Y)
    print(f'{i:2d} loss={loss.item()}')
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    for p in parameters:
        p.data += -0.1 * p.grad

 0 loss=10.709585189819336
 1 loss=10.40762996673584
 2 loss=10.127808570861816
 3 loss=9.864365577697754
 4 loss=9.61450481414795
 5 loss=9.37644100189209
 6 loss=9.148946762084961
 7 loss=8.931112289428711
 8 loss=8.722232818603516
 9 loss=8.52175235748291


In [180]:
# each pass taking a while, lets do mini batches now
for i in range(10000):

    # minibatch construct
    ix = torch.randint(0, X.shape[0], (32,))
    
    # forward pass
    emb = C[X[ix]] # 32x3x2
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # -1 means torch will infer the size - output is 32x100
    logits = h @ W2 + b2 # 32x27
    loss = F.cross_entropy(logits, Y[ix])
    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    for p in parameters:
        p.data += -.01 * p.grad
print(f'{i:2d} loss={loss.item()}')

9999 loss=2.192488431930542


In [181]:
# evaluate loss for entire dataset
emb = C[X]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Y)
loss.item()

2.3079402446746826